In [1]:
# Importing GemPy
import gempy as gp

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from gempy.plot import visualization_2d as vv
from gempy.plot import vista

First we create a model to store things in.

In [2]:
gp.create_model('From_Fatiando')

From_Fatiando  2020-06-06 18:59

We want to import the data from disk, where it is stored as a netCDF file.

In [3]:
import xarray as xr

In [5]:
fat_grid = xr.open_dataset('../data/fatiando_gridded_data.nc')
fat_grid

<xarray.Dataset>
Dimensions:           (easting: 95, northing: 157)
Coordinates:
    extra_coord       (northing, easting) float64 ...
  * northing          (northing) float64 4.175e+06 4.176e+06 ... 4.253e+06
  * easting           (easting) float64 -3.24e+05 -3.235e+05 ... -2.769e+05
Data variables:
    magnetic_anomaly  (northing, easting) float64 ...
Attributes:
    metadata:  Generated by EQLHarmonic(damping=1, relative_depth=1000)

The model needs some extents, in the format `[x_min, x_max, y_min, y_max, z_min, z_max]`.

We want to use the extents from the fatiando grid, which we can extract thus. The z coords are a little weird, so I am guessing them for now. This would presumably be how deep you expect your model to actually extend. For mag data, a couple of km seems reasonable for now, but we want to vertically exaggerate things, in this case by a factor of 50.

In [6]:
extent = [fat_grid.northing.to_series().min(), fat_grid.northing.to_series().max(),
          fat_grid.easting.to_series().min(), fat_grid.easting.to_series().max(),
          fat_grid.magnetic_anomaly.to_series().min(), fat_grid.magnetic_anomaly.to_series().max() * 50]

In [7]:
geo_model = gp.create_model('Model1')
geo_model = gp.init_data(geo_model, extent=extent, resolution=[100, 100, 10])

Active grids: ['regular']


In [8]:
gp.set_interpolator(geo_model, theano_optimizer='fast_compile', verbose=[])

Setting kriging parameters to their default values.
Compiling theano function...


C:\Users\mtb\Miniconda3\envs\gempy-test\lib\site-packages\theano\scan_module\scan_perform_ext.py:76: UserWarning: The file scan_perform.c is not available. This donot happen normally. You are probably in a strangesetup. This mean Theano can not use the cython code for scan. If youwant to remove this warning, use the Theano flag'cxx=' (set to an empty string) to disable all ccode generation.
  "The file scan_perform.c is not available. This do"
C:\Users\mtb\Miniconda3\envs\gempy-test\lib\site-packages\theano\scan_module\scan_perform_ext.py:76: UserWarning: The file scan_perform.c is not available. This donot happen normally. You are probably in a strangesetup. This mean Theano can not use the cython code for scan. If youwant to remove this warning, use the Theano flag'cxx=' (set to an empty string) to disable all ccode generation.
  "The file scan_perform.c is not available. This do"
C:\Users\mtb\Miniconda3\envs\gempy-test\lib\site-packages\theano\scan_module\scan_perform_ext.py:76: Use

Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                       values
range                94110.3
$C_o$            2.10875e+08
drift equations          [3]


This seems to give something reasonably shaped.

In [ ]:
p3d = gp.plot_3d(geo_model)

Our next issue is to add points and surfaces. In theory, we can use each axis to create the points, but that will almost certainly be too many.

GemPy expects to be given a `DataFrame`, not a `Dataset`, so we will need to convert it. It also expects to find the columns `X|x`, `Y|y`, `Z|z` and one of `surface`, `Surface`, `surfaces`, `surfaces`, `formations`, or `formation`, which probably do not exist from harmonica by default.

Note: being able to set this might be nice if I am importing a cube of points.

In [9]:
df_fat_grid = fat_grid.to_dataframe()
df_fat_grid.reset_index(inplace=True)
new_columns = {'easting': 'X',
               'northing': 'Y',
              }
df_fat_grid.rename(columns=new_columns, inplace=True)
df_fat_grid['surface'] = 'mag_surface'
df_fat_grid['Z'] = df_fat_grid['magnetic_anomaly'] * 50

df_fat_grid

,X,Y,magnetic_anomaly,extra_coord,surface,Z
0,-324002.139135,4.175068e+06,30.667677,1500.0,mag_surface,1533.383839
1,-324002.139135,4.175568e+06,30.405465,1500.0,mag_surface,1520.273258
2,-324002.139135,4.176067e+06,30.265085,1500.0,mag_surface,1513.254228
3,-324002.139135,4.176567e+06,30.310932,1500.0,mag_surface,1515.546619
4,-324002.139135,4.177067e+06,30.669662,1500.0,mag_surface,1533.483114
...,...,...,...,...,...,...
14910,-276890.268483,4.251041e+06,172.457827,1500.0,mag_surface,8622.891370
14911,-276890.268483,4.251541e+06,168.851813,1500.0,mag_surface,8442.590645
14912,-276890.268483,4.252040e+06,163.476307,1500.0,mag_surface,8173.815338
14913,-276890.268483,4.252540e+06,156.952806,1500.0,mag_surface,7847.640282


We need to decimate the data somehow, so that we have fewer points.

In [10]:
dec_df_fat_grid = df_fat_grid.iloc[::150].reset_index()

In [11]:
geo_model.set_surface_points(dec_df_fat_grid)

,X,Y,Z,smooth,surface
0,-324002.139135,4.175068e+06,1533.383839,0.000002,mag_surface
1,-324002.139135,4.250041e+06,-5365.022287,0.000002,mag_surface
2,-323500.949022,4.246542e+06,-5800.937793,0.000002,mag_surface
3,-322999.758908,4.243044e+06,-6190.410350,0.000002,mag_surface
4,-322498.568795,4.239545e+06,-6676.778121,0.000002,mag_surface
5,-321997.378682,4.236046e+06,-6792.256819,0.000002,mag_surface
6,-321496.188568,4.232547e+06,-7014.618490,0.000002,mag_surface
7,-320994.998455,4.229049e+06,-7342.856694,0.000002,mag_surface
8,-320493.808342,4.225550e+06,-7885.518557,0.000002,mag_surface
9,-319992.618229,4.222051e+06,-8713.366590,0.000002,mag_surface


In [12]:
geo_model.add_surfaces(['basement'])

,surface,series,order_surfaces,color,id
0,mag_surface,Default series,1,#015482,1
1,basement,Default series,2,#9f0052,2


In [13]:
gp.compute_model(geo_model)

NotImplementedError: We didn't implemented yet the case where scan do 0 iteration
Apply node that caused the error: for{cpu,Looping}(Elemwise{minimum,no_inplace}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, IncSubtensor{Set;:int64:}.0, Number of points per surface used to split rest-ref, fault relation matrix, <TensorType(float64, scalar)>, <TensorType(float64, scalar)>, Range, Covariance at 0, <TensorType(float64, scalar)>, Nugget effect of gradients, Nugget effect of scalar, Atten

The above error seems to be from placing points outside the plotting area.

In [ ]:
p3d = gp.plot_3d(geo_model)